In [1]:

import pandas as pd

In [13]:
from datasets import load_dataset

dataset = load_dataset("MonoHime/ru_sentiment_dataset")

c:\Users\nklen\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 189891 examples [00:08, 21778.62 examples/s]
Generating validation split: 21098 examples [00:00, 25364.27 examples/s]


In [2]:
data = pd.read_csv('datasets.csv')

In [3]:
print(data.head())

   Unnamed: 0                                               text  sentiment
0       43956  Развода на деньги нет\nНаблюдаюсь в Лайфклиник...          1
1       17755  Отель выбрали потому что рядом со стадионом. О...          0
2       20269  Вылечили\nГноился с рождения глазик, в поликли...          1
3       16648  Хорошее расположение.С вокзала дошли пешком.Но...          0
4       27879  Отличное месторасположение,прекрасный вид,особ...          1


In [11]:
data.shape

(210989, 3)

In [5]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

messages = [
    'Сегодня хорошая погода',
    'Мне нравится эта задача',
    'Вы очень глупы!',
    'Сколько стоит эти кросовки?'
]

results = model.predict(messages, k=2)
for message, sentiment in zip(messages, results):
    print(message, '-&gt;', sentiment)


Сегодня хорошая погода -&gt; {'positive': 0.9879562258720398, 'negative': 0.01323282066732645}
Мне нравится эта задача -&gt; {'positive': 0.8175845146179199, 'neutral': 0.5000100135803223}
Вы очень глупы! -&gt; {'negative': 0.33459946513175964, 'neutral': 0.19194278120994568}
Сколько стоит эти кросовки? -&gt; {'neutral': 0.9688661694526672, 'negative': 0.06372499465942383}


In [4]:

import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted


/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
d = 'я тебя люблю!'
print(d)
print(predict(d))
print(tokenizer.tokenize(d))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(d)))

я тебя люблю!
[1]
['я', 'тебя', 'люблю', '!']
[358, 1425, 3852, 106]


In [6]:
sentences = list(data.text)
print(sentences[0:5])

['Развода на деньги нет\nНаблюдаюсь в Лайфклиник по беременности, развода на деньги нет, врачи не плохие, по поводу ресепшен соглашусь (путают документы).\n', 'Отель выбрали потому что рядом со стадионом. Отель 4*. Номер большой. Кровать 2-спальная одна. 2 одеяла. Много подушек. Есть зона отдыха. Чайный сет. 2 бутылки по 0,5 л бесплатно каждый день. Много шкафов. Мини-бар. Утюг и гладильная доска, отдельно прибор для глажки брюк. Санузел общий большой. Ванна, лейка съемная. 2 раковины, фен. Халаты, тапочки. Расширенный пакет косметических средств, даже соль для ванны. Интернет быстрый. Так как этаж высокий был, вид из окна на город. Один недостаток в номере 919: он напротив хозлифта и с утра начинает пользоваться им персонал для обслуживания завтрака на 9 этаже. Очень слышно звон посуды и разговоры. Завтрак не включен, стоит 23 евро. Рядом с отелем есть кафе и subway.', 'Вылечили\nГноился с рождения глазик, в поликлинике назначили лечение, не помогло сходили сюда вылечили (офтальмолог 

In [7]:
labels = data.sentiment.values
print(labels)

[1 0 1 ... 0 0 1]


In [8]:
print(sentences[0])

Развода на деньги нет
Наблюдаюсь в Лайфклиник по беременности, развода на деньги нет, врачи не плохие, по поводу ресепшен соглашусь (путают документы).



In [7]:
max_len = 0
id_max = 0
counts = 0
    
for sent in sentences:

    input_ids = tokenizer.encode(sent, add_special_tokens = True)

    if max_len < len(input_ids):
        id_max = sent
        max_len = max(max_len, len(input_ids))
    if len(input_ids)  <= 510:
        
        counts += 1   
    

In [40]:
for i in range(len(data)):
    if (len(data.text[i]) > 510):
        data.drop(i)


KeyboardInterrupt: 

In [9]:

print(len(data.text[0])  < 500)

True


In [12]:
print(max_len)
print(id_max)
print(counts)

NameError: name 'max_len' is not defined

In [10]:
input_ids = []
attention_mask = []

for sent in sentences:

    encoded_dict = tokenizer.encode_plus(
        sent, 
        add_special_tokens = True, 
        max_length = 512, 
        pad_to_max_length = True,
        return_attention_mask = True, 
        return_tensors = 'pt', 
    )

    

    attention_mask.append(encoded_dict['attention_mask'])
    
    input_ids.append(encoded_dict['input_ids'])
    #print(input_ids)

input_ids = torch.cat(input_ids, dim=0)
attention_mask = torch.cat(attention_mask, dim = 0)    

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
print(type(input_ids[2]))
print(len(attention_mask))

<class 'torch.Tensor'>
210989


In [14]:
input_ids = torch.cat(input_ids, dim=0)
attention_mask = torch.cat(attention_mask, dim = 0) 

TypeError: cat() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [12]:
labels = torch.tensor(labels)

In [13]:
print(type(labels))

<class 'torch.Tensor'>


In [14]:
print(sentences[0], input_ids[0])

Развода на деньги нет
Наблюдаюсь в Лайфклиник по беременности, развода на деньги нет, врачи не плохие, по поводу ресепшен соглашусь (путают документы).
 tensor([  101, 52317,   819,   801,  1928,  1036, 30141, 15230,   862,   340,
        69781, 98910,  1520,   797, 31050,   128, 39565,   801,  1928,  1036,
          128, 10743,   802, 11830,   128,   797,  3502, 60358,  2486, 16245,
          120, 50896,  6994,   122,   132,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,    

In [15]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_mask, labels)

train_size = int(0.9 * len(dataset))
val_size  = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) 

print(train_size)
print(val_size)

189890
21099


In [16]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
) 

validation_dataloader = DataLoader(
    val_dataset, 
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size   
)



In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig


model = BertForSequenceClassification.from_pretrained(
    "blanchefort/rubert-base-cased-sentiment",
    num_labels = 3, 
    output_attentions = False,
    output_hidden_states = False,
)


#model.mps()

In [18]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8,   
                  )

/Users/nikitaklenskij/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
from transformers import get_linear_schedule_with_warmup

epochs  = 4

total_steps  = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, 
                                            num_training_steps=total_steps)

Training Loop

In [20]:
import numpy as np

In [21]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, asix = 1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat ==  labels_flat) / len(labels_flat)

In [22]:
import random
import time

In [24]:
training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):
    print("")
    print("=========== Epovh {:} / {:} ===========".format(epoch_i, epochs))
    print('Training...')

    t0 = time.time()


    total_train_loss = 0 

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)


            print('  Batch {: >5,}   of   {: >5,}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))


        b_input_ids = batch[0].to('cpu')
        b_input_mask = batch[1].to('cpu')
        b_labels = batch[2].to('cpu')

        model.zero_grad()

        res = model(b_input_ids, 
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels
                    )

        loss = res['loss']
        logits = res['logits']

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


        optimizer.step()

        scheduler.step()

    
    avg_train_loss = avg_train_loss / len(train_dataloader)


    training_time = format_time(time.time() - t0)


    print('')
    print('   Average training loss: {0:.2f}'.format(avg_train_loss))
    print('   Trainning epoch took: {:}'.format(training_time))


    print('')
    print('Running Validation...')

    to = time.time()


    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0 
    nb_eval_steps = 0


    for batch in validation_dataloader:


        b_input_ids = batch[0].to('cpu')
        b_input_mask = batch[1].to('cpu')
        b_labels = batch[2].to('cpu')

        with tourc.no_grad():
            

            res = model(b_input_ids, 
                    token_type_ids=None,
                    attention_mask=input_mask,
                    labels=b_labels
                    )

            loss = res['loss']
            logits = res['logits']


        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()


        total_eval_accuracy += flat_accuracy(logits, label_ids)


    avg_val_accuracy = total_eval_accuracy() / len(validation_dataloader)

    print('    Accuracy: {0:.2f}'.format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print('    Validation Loss: {2:.2f}'.format(avg_val_loss))
    print('    Validation tok: {2:.2f}'.format(validation_time))


    training_stats.append(
        {
            'epoch' : epoch_i + 1,
            'Trainig Loss' : avg_train_loss,
            'Valid loss' : avg_val_loss,
            'Valid Accur' : avg_val_accuracy,
            'Training Time' : training_time, 
            'Valid Time' : validation_time
        }
    )



print('')
print('Training complete!')


print('Total training took {:} (hh:mm:ss)'.format(format_time(time.time() - total_t0)))













=========== Epovh 0 / 4 ===========
Training...


KeyboardInterrupt: 

In [ ]:
m = [[1,0], [5, 1], [1, 1], [1, 0], [0, 0], [9, 6], [3, 5]] 

m.sort()

print(m)